# Grab transit routes near the State Highway Network (SHN)

* Find transit routes within 1 mile of the SHN.
* Visualize from operator or highway point of view to see % parallel or intersecting

Data Sources: 
* [SHN on Geoportal](https://opendata.arcgis.com/datasets/77f2d7ba94e040a78bfbe36feb6279da_0.geojson) > processed in `create_parallel_corridors.py` > exported to GCS > saved in catalog.
* Transit routes: all transit routes, those in `gtfs_schedule.shapes` and those not, but found in `stops`. Use `traffic_ops/export_shapefiles.py` created `routes_assembled.parquet` in GCS > saved in catalog.

In [ ]:
import branca
import geopandas as gpd
import intake
import pandas as pd

import create_parallel_corridors
from shared_utils import geography_utils, map_utils, utils
from shared_utils import calitp_color_palette as cp

catalog = intake.open_catalog("*.yml")

IMG_PATH = create_parallel_corridors.IMG_PATH
DATA_PATH = create_parallel_corridors.DATA_PATH

## Read in processed data 

The overlay/intersection between transit routes and highways:
* % of transit route that intersection represents
* % of highway that intersection represents

If it passes a certain threshold for transit route (> 0.5?) **and** certain threshold for highway (> 0.1?), then count that transit route as parallel. Otherwise, intersecting.

Play with thresholds and see what makes sense, visualize with interactive maps.

Look at LA Metro, San Jose specifically to see if it is behaving as expected.

In [ ]:
'''
create_parallel_corridors.make_analysis_data(hwy_buffer_feet=
                       shared_utils.geography_utils.FEET_PER_MI, 
                       pct_route_threshold = 0.5,
                       pct_highway_threshold = 0.1,
                       DATA_PATH 
                      )


# For map, need highway to be 250 ft buffer
#highways = create_parallel_corridors.process_highways(buffer_feet=250)
#highways.to_parquet(f"{DATA_PATH}highways.parquet")
'''

## Map 

In [ ]:
gdf = gpd.read_parquet(f"{DATA_PATH}parallel_or_intersecting.parquet")
highways= gpd.read_parquet(f"{DATA_PATH}highways.parquet")

In [ ]:
def data_to_plot(df):
    keep_cols = ["itp_id", "route_id", 
                 "Route", "County", "District", "RouteType",
                 "pct_route", "pct_highway", "parallel",
                 "geometry"
                ]
    df = df[keep_cols].reset_index(drop=True)
    df = df.assign(
        geometry = df.geometry.buffer(200).simplify(tolerance=100),
    )
    
    # Use simplify to make gdf smaller
    # folium map is creating too large of an HTML file to check in
    
    return df


In [ ]:
hwys_popup_dict = {
    "Route": "Highway Route",
    "RouteType": "Route Type",
    "County": "County"   
}

transit_popup_dict = {
    "itp_id": "Operator ITP ID",
    "route_id": "Route ID",
    "pct_route": "% overlapping route",
    "pct_highway": "% overlapping highway",
}

hwys_color = branca.colormap.StepColormap(
    colors=["black", "gray"],
)

colorscale = branca.colormap.StepColormap(
    colors=[
        cp.CALITP_CATEGORY_BRIGHT_COLORS[0], #blue
        cp.CALITP_CATEGORY_BRIGHT_COLORS[1] # orange
    ],
)

In [ ]:
# Save this colorscale as png and use as FloatImage in folium map
print("           Intersecting                        Parallel")
display(colorscale)

In [ ]:
MAP_ME = {
    182: "LA Metro", 
    294: "SJ Valley Transportation Authority", 
    279: "BART", 
    282: "SF Muni",
    278: "SD Metropolitan Transit System", 
}


for itp_id, operator in MAP_ME.items(): 
    
    transit_df = data_to_plot(gdf[gdf.itp_id==itp_id])
    highway_df = (highways[highways.County.isin(transit_df.County)]
                  .reset_index(drop=True)
                 )
    
    # Instead of using county centroid, calculate centroid from transit_df
    # Otherwise, it's too zoomed out from where transit routes are
    transit_centroid = (transit_df
                        .to_crs(geography_utils.WGS84).geometry.centroid
                        .iloc[0]
                       )

    LAYERS_DICT = {
        "Highways": {"df": highway_df,
            "plot_col": "Route",
            "popup_dict": hwys_popup_dict, 
            "tooltip_dict": hwys_popup_dict,
            "colorscale": hwys_color,
        },
        "Transit Routes": {"df": transit_df,
            "plot_col": "parallel",
            "popup_dict": transit_popup_dict, 
            "tooltip_dict": transit_popup_dict,
            "colorscale": colorscale,
        },
    }
    
    LEGEND_URL = (
        "https://raw.githubusercontent.com/cal-itp/data-analyses/"
        "main/bus_service_increase/"
        "img/legend_intersecting_parallel.png"
    )
    
    LEGEND_DICT = {
        "legend_url": LEGEND_URL,
        "legend_bottom": 85,
        "legend_left": 5,
    }
    
    
    fig = map_utils.make_folium_multiple_layers_map(
        LAYERS_DICT,
        fig_width = 700, fig_height = 700, 
        zoom=11, 
        centroid = [round(transit_centroid.y,2), 
                    round(transit_centroid.x, 2)], 
        title=f"Parallel vs Intersecting Lines for {operator}",
        legend_dict = LEGEND_DICT
    )

    fig.save(f"{IMG_PATH}parallel_{itp_id}.html")
    print(f"{itp_id} map saved")

In [ ]:
def make_folium_multiple_layers_map(LAYERS_DICT, fig_width, fig_height, 
                                    zoom=map_utils.REG_CENTROIDS["CA"]["zoom"], 
                                    centroid = REGION_CENTROIDS["CA"]["centroid"], 
                                    title="Chart Title",
                                   ):
    
    # Pass more kwargs through various sub-functions
    # https://stackoverflow.com/questions/26534134/python-pass-different-kwargs-to-multiple-functions
    
    fig = Figure(width = fig_width, height = fig_height,)
    
    
    m = folium.Map(location=centroid, tiles='cartodbpositron', 
               zoom_start=zoom, width=fig_width,height=fig_height,)
    
    title_html = f'''
         <h3 align="center" style="font-size:20px"><b>{title}</b></h3>
         '''
    
    fig.get_root().html.add_child(folium.Element(title_html))
    
    # Define function that can theoretically pop out as many polygon layers as needed
    def get_layer(df, plot_col, popup_dict, tooltip_dict, 
              colorscale, layer_name, **kwargs):    

        popup = map_utils.format_folium_popup(popup_dict)
        tooltip = map_utils.format_folium_tooltip(tooltip_dict)

        #https://medium.com/analytics-vidhya/create-and-visualize-choropleth-map-with-folium-269d3fd12fa0
        
        g = folium.GeoJson(
            df,
            style_function=lambda x: {
                "fillColor": colorscale(x["properties"][plot_col])
                if x["properties"][plot_col] is not None
                else f"gray",
                "color": "#FFFFFF",
                "fillOpacity": 0.8,
                "weight": 0.2,
            },
            tooltip=tooltip,
            popup=popup,
            name = layer_name,
        )

        return g

    # Now, loop through the keys in the LAYERS_DICT,
    # Unpack the dictionary associated with each layer,
    # Then attach that layer to the Map element
    for key, nested_dict in LAYERS_DICT.items():
        # key: layer name or layer number
        # value: dictionary of all the components associated with folium layer
        d = nested_dict
        layer = get_layer(df = d["df"], plot_col = d["plot_col"], 
                          popup_dict = d["popup_dict"], 
                          tooltip_dict = d["tooltip_dict"], 
                          colorscale = d["colorscale"], 
                          layer_name = key,
                         )
        layer.add_to(m)
    
    
    legend_url = (
        "https://raw.githubusercontent.com/cal-itp/data-analyses/"
        "more-highways/bus_service_increase/"
        "img/legend_intersecting_parallel.png"
    )

    legend_html = f"""

       <style> #background_img {
                position : absolute;
                background:url('{legend_url}');
                width : 16.9%;
                height: 17.7%;
                right: 20px;
                bottom: 50px;
                z-index: 99;
                background-repeat: no-repeat;
                background-size: contain; 
                background-color: white;
                opacity: 0.85;
                font-size: 12px;
                font-weight: bold;
                }
            </style>

            <div id="background_img" class="backgroundimg" ></div>
            """
    m.get_root().html.add_child(folium.Element(legend_html))
    
    folium.LayerControl('topright', collapsed=False).add_to(m)

    
    # Now, attach everything to Figure    
    fig.add_child(m)
    
    return fig

In [ ]:
#https://stackoverflow.com/questions/61065945/how-can-a-plot-a-map-with-folium-and-place-a-bar-graph-next-to-the-map-in-python
# Can folium also include subplots for other stats?